
# preprocessing.ipynb 기준 공행성 쌍 예측 모델

이 노트북은 `preprocessing.ipynb`에서 생성한 전처리 결과를 사용하여 공행성 쌍을 예측합니다.

## 특징
- preprocessing.ipynb의 df_panel 사용
- 시계열 특성 (이동평균, 변화율) 활용
- 공급망 정보 활용


In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge, QuantileRegressor
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm

print("라이브러리 로드 완료")


라이브러리 로드 완료


## 1. 전처리된 데이터 준비

주의: preprocessing.ipynb를 먼저 실행하여 df_panel을 준비해야 합니다.


In [2]:
# preprocessing.ipynb 방식으로 전처리 (간단 버전)
# 실제로는 preprocessing.ipynb를 실행한 후 df_panel을 사용해야 함

train = pd.read_csv('../data/train.csv')

# date 컬럼 생성
train['date'] = pd.to_datetime(
    train['year'].astype(str) + "-" + train['month'].astype(str).str.zfill(2) + "-01"
)

# 월별 집계
agg_cols = ["value", "weight", "quantity"]
df_train_monthly = (
    train.groupby(["item_id", "hs4", "date"], as_index=False)[agg_cols].sum()
)

# Panel 생성
min_month = df_train_monthly["date"].min()
max_month = df_train_monthly["date"].max()
full_months = pd.date_range(min_month, max_month, freq="MS")
all_items = df_train_monthly["item_id"].unique()

panel = pd.MultiIndex.from_product(
    [all_items, full_months],
    names=["item_id", "date"]
).to_frame(index=False)

df_panel = (
    panel.merge(df_train_monthly, on=["item_id", "date"], how="left")
         .sort_values(["item_id", "date"])
         .reset_index(drop=True)
)

# NaN을 0으로 채우기
for c in agg_cols:
    df_panel[c] = df_panel[c].fillna(0)

df_panel["year"] = df_panel["date"].dt.year
df_panel["month"] = df_panel["date"].dt.month

# HS4 매핑 보완
item_hs4_mapping = (
    df_panel[df_panel['hs4'].notna()]
    .groupby('item_id')['hs4']
    .first()
    .to_dict()
)
df_panel['hs4'] = df_panel['item_id'].map(item_hs4_mapping).fillna(df_panel['hs4'])

# 공급망 정보 추가
def get_supply_chain_tier(hs4):
    if pd.isna(hs4):
        return 0
    hs4_int = int(hs4)
    if 2800 <= hs4_int <= 3899:
        return 1  # Tier 1
    elif 7200 <= hs4_int <= 8399:
        return 2  # Tier 2
    elif 8400 <= hs4_int <= 8599:
        return 3  # Tier 3
    else:
        return 0

df_panel['supply_chain_tier'] = df_panel['hs4'].apply(get_supply_chain_tier)

# 시계열 특성 추가
df_panel = df_panel.sort_values(['item_id', 'date']).reset_index(drop=True)
df_panel['value_ma6'] = df_panel.groupby('item_id')['value'].transform(
    lambda x: x.rolling(window=6, min_periods=1).mean()
)
# 변화율 계산 (inf, -inf 처리)
df_panel['value_pct_change'] = df_panel.groupby('item_id')['value'].pct_change().fillna(0)
# inf, -inf 값을 0으로 대체 (0에서 나누기로 인한 inf 발생 방지)
df_panel['value_pct_change'] = df_panel['value_pct_change'].replace([np.inf, -np.inf], 0)

print(f"전처리 완료: {len(df_panel):,}행")
df_panel.head()


전처리 완료: 4,300행


,item_id,date,hs4,value,weight,quantity,year,month,supply_chain_tier,value_ma6,value_pct_change
0,AANGBULD,2022-01-01,4810.0,14276.0,17625.0,0.0,2022,1,0,14276.000000,0.000000
1,AANGBULD,2022-02-01,4810.0,52347.0,67983.0,0.0,2022,2,0,33311.500000,2.666783
2,AANGBULD,2022-03-01,4810.0,53549.0,69544.0,0.0,2022,3,0,40057.333333,0.022962
3,AANGBULD,2022-04-01,4810.0,0.0,0.0,0.0,2022,4,0,30043.000000,-1.000000
4,AANGBULD,2022-05-01,4810.0,26997.0,34173.0,0.0,2022,5,0,29433.800000,0.000000


## 2. Pivot 테이블 생성


In [3]:
# Pivot 테이블 생성 (value 사용)
pivot = (
    df_panel.pivot_table(
        index='item_id',
        columns='date',
        values='value',
        aggfunc='sum',
        fill_value=0.0
    )
)

print(f"Pivot table shape: {pivot.shape}")

# item_id-hs4 매핑
item_hs4 = df_panel[['item_id', 'hs4']].drop_duplicates().set_index('item_id')['hs4'].to_dict()

pivot.head()


Pivot table shape: (100, 43)


date,2022-01-01,2022-02-01,2022-03-01,2022-04-01,2022-05-01,2022-06-01,2022-07-01,2022-08-01,2022-09-01,2022-10-01,...,2024-10-01,2024-11-01,2024-12-01,2025-01-01,2025-02-01,2025-03-01,2025-04-01,2025-05-01,2025-06-01,2025-07-01
item_id,,,,,,,,,,,,,,,,,,,,,
AANGBULD,14276.0,52347.0,53549.0,0.0,26997.0,84489.0,0.0,0.0,0.0,0.0,...,428725.0,144248.0,26507.0,25691.0,25805.0,0.0,38441.0,0.0,441275.0,533478.0
AHMDUILJ,242705.0,120847.0,197317.0,126142.0,71730.0,149138.0,186617.0,169995.0,140547.0,89292.0,...,123085.0,143451.0,78649.0,125098.0,80404.0,157401.0,115509.0,127473.0,89479.0,101317.0
ANWUJOKX,0.0,0.0,0.0,63580.0,81670.0,26424.0,8470.0,0.0,0.0,80475.0,...,0.0,0.0,0.0,27980.0,0.0,0.0,0.0,0.0,0.0,0.0
APQGTRMF,383999.0,512813.0,217064.0,470398.0,539873.0,582317.0,759980.0,216019.0,537693.0,205326.0,...,683581.0,2147.0,0.0,25013.0,77.0,20741.0,2403.0,3543.0,32430.0,40608.0
ATLDMDBO,143097177.0,103568323.0,118403737.0,121873741.0,115024617.0,65716075.0,146216818.0,97552978.0,72341427.0,87454167.0,...,60276050.0,30160198.0,42613728.0,64451013.0,38667429.0,29354408.0,42450439.0,37136720.0,32181798.0,57090235.0


## 3. 공행성쌍 탐색


In [4]:
def safe_corr(x, y):
    if np.std(x) == 0 or np.std(y) == 0:
        return 0.0
    return float(np.corrcoef(x, y)[0, 1])


def calculate_stability_score(x, y, lag):
    """시계열 안정성 점수 계산 (변동성 낮을수록 높은 점수)"""
    if len(x) <= lag or len(y) <= lag:
        return 0.0
    
    x_aligned = x[:-lag]
    y_aligned = y[lag:]
    
    # 0이 아닌 값만 사용
    x_nonzero = x_aligned[x_aligned != 0]
    y_nonzero = y_aligned[y_aligned != 0]
    
    if len(x_nonzero) < 3 or len(y_nonzero) < 3:
        return 0.0
    
    # 변동계수 (CV) 계산: 표준편차 / 평균
    cv_x = np.std(x_nonzero) / (np.mean(x_nonzero) + 1e-8)
    cv_y = np.std(y_nonzero) / (np.mean(y_nonzero) + 1e-8)
    
    # 변동계수가 낮을수록 안정적 (점수 높음)
    stability = 1.0 / (1.0 + (cv_x + cv_y) / 2.0)
    return min(stability, 1.0)


# 공급망 계층 함수 (Cell 3에서 정의된 함수 재정의)
def get_supply_chain_tier(hs4):
    """HS4 코드를 기반으로 공급망 계층 반환"""
    if pd.isna(hs4) or hs4 == 0:
        return 0
    try:
        hs4_int = int(hs4)
        if 2800 <= hs4_int <= 3899:
            return 1  # Tier 1
        elif 7200 <= hs4_int <= 8399:
            return 2  # Tier 2
        elif 8400 <= hs4_int <= 8599:
            return 3  # Tier 3
        else:
            return 0
    except (ValueError, TypeError):
        return 0


def find_comovement_pairs(pivot, item_hs4, max_lag=6, min_nonzero=12, 
                          corr_threshold=0.4, min_abs_corr=0.3, max_pairs=None):
    """공행성쌍 탐색 (preprocessing.ipynb 방식)"""
    items = pivot.index.to_list()
    months = pivot.columns.to_list()
    n_months = len(months)

    results = []

    for i, leader in tqdm(enumerate(items), desc="Finding comovement pairs"):
        leader_hs4 = item_hs4.get(leader, 0)
        x = pivot.loc[leader].values.astype(float)
        if np.count_nonzero(x) < min_nonzero:
            continue

        for follower in items:
            if follower == leader:
                continue

            follower_hs4 = item_hs4.get(follower, 0)
            y = pivot.loc[follower].values.astype(float)
            if np.count_nonzero(y) < min_nonzero:
                continue

            best_lag = None
            best_corr = 0.0
            best_stability = 0.0

            # lag = 1 ~ max_lag 탐색
            for lag in range(1, max_lag + 1):
                if n_months <= lag:
                    continue
                corr = safe_corr(x[:-lag], y[lag:])
                if abs(corr) > abs(best_corr):
                    best_corr = corr
                    best_lag = lag
                    best_stability = calculate_stability_score(x, y, lag)

            # 이중 필터링: 최소 상관계수 + 임계값
            if best_lag is not None and abs(best_corr) >= min_abs_corr and abs(best_corr) >= corr_threshold:
                # 공급망 정보 추가
                leader_tier = get_supply_chain_tier(leader_hs4)
                follower_tier = get_supply_chain_tier(follower_hs4)
                same_hs4 = 1 if leader_hs4 == follower_hs4 else 0
                
                # 공급망 방향성 점수
                if leader_hs4 == follower_hs4:
                    supply_chain_score = 1.5
                elif leader_tier < follower_tier:
                    supply_chain_score = 2.0
                elif leader_tier == follower_tier:
                    supply_chain_score = 1.0
                else:
                    supply_chain_score = 0.5

                # 종합 점수: 상관계수 + 안정성 + 공급망 점수
                composite_score = abs(best_corr) * 0.5 + best_stability * 0.3 + (supply_chain_score / 2.0) * 0.2

                results.append({
                    "leading_item_id": leader,
                    "following_item_id": follower,
                    "best_lag": best_lag,
                    "max_corr": best_corr,
                    "stability_score": best_stability,
                    "composite_score": composite_score,
                    "supply_chain_score": supply_chain_score,
                    "same_hs4": same_hs4,
                })

    pairs = pd.DataFrame(results)
    
    # 종합 점수로 정렬 후 상위 N개 선택 (선택사항)
    if max_pairs is not None and len(pairs) > max_pairs:
        pairs = pairs.nlargest(max_pairs, 'composite_score')
    
    return pairs


# 개선 1: max_pairs 튜닝으로 Precision 최적화
# 종합 점수로 정렬 후 상위 N개만 선택 (Precision 향상)
MAX_PAIRS = None  # 튜닝 가능: 1200, 1500, 1800 등
pairs = find_comovement_pairs(pivot, item_hs4, max_pairs=MAX_PAIRS)
print(f"탐색된 공행성쌍 수: {len(pairs)}")
if MAX_PAIRS is not None:
    print(f"  (종합 점수 상위 {MAX_PAIRS}개 선택)")
pairs.head()



Finding comovement pairs: 0it [00:00, ?it/s]


Finding comovement pairs: 7it [00:00, 66.78it/s]


Finding comovement pairs: 14it [00:00, 59.77it/s]


Finding comovement pairs: 21it [00:00, 57.85it/s]


Finding comovement pairs: 27it [00:00, 57.44it/s]

...


Finding comovement pairs: 93it [00:01, 61.53it/s]


Finding comovement pairs: 100it [00:01, 60.11it/s]


Finding comovement pairs: 100it [00:01, 62.06it/s]

탐색된 공행성쌍 수: 1425


,leading_item_id,following_item_id,best_lag,max_corr,stability_score,composite_score,supply_chain_score,same_hs4
0,AANGBULD,APQGTRMF,5,-0.443984,0.478776,0.565625,2.0,0
1,AANGBULD,DEWLVASR,6,0.640221,0.497137,0.669251,2.0,0
2,AANGBULD,DNMPSKTB,4,-0.410635,0.558690,0.572924,2.0,0
3,AANGBULD,EVBVXETX,6,0.436623,0.581733,0.592832,2.0,0
4,AANGBULD,FTSVTTSR,3,0.531400,0.430149,0.494745,1.0,0


## 4. 학습 데이터 생성


In [5]:
def build_training_data(pivot, pairs, df_panel):
    """preprocessing.ipynb의 시계열 특성을 활용한 학습 데이터 생성"""
    months = pivot.columns.to_list()
    n_months = len(months)

    rows = []

    for row in tqdm(pairs.itertuples(index=False), desc="Building training data", total=len(pairs)):
        leader = row.leading_item_id
        follower = row.following_item_id
        lag = int(row.best_lag)
        corr = float(row.max_corr)
        supply_chain_score = float(row.supply_chain_score)
        same_hs4 = int(row.same_hs4)

        if leader not in pivot.index or follower not in pivot.index:
            continue

        a_series = pivot.loc[leader].values.astype(float)
        b_series = pivot.loc[follower].values.astype(float)

        # t+1이 존재하고, t-lag >= 0인 구간만 학습에 사용
        for t in range(max(lag, 1), n_months - 1):
            b_t = b_series[t]
            b_t_1 = b_series[t - 1]
            a_t_lag = a_series[t - lag]
            b_t_plus_1 = b_series[t + 1]

            # 시계열 특성 추가
            current_date = months[t]
            try:
                leader_features = df_panel[
                    (df_panel['item_id'] == leader) & (df_panel['date'] == current_date)
                ]
                follower_features = df_panel[
                    (df_panel['item_id'] == follower) & (df_panel['date'] == current_date)
                ]

                leader_ma6 = leader_features['value_ma6'].values[0] if len(leader_features) > 0 and 'value_ma6' in leader_features.columns else 0
                follower_ma6 = follower_features['value_ma6'].values[0] if len(follower_features) > 0 and 'value_ma6' in follower_features.columns else 0
                follower_pct_change = follower_features['value_pct_change'].values[0] if len(follower_features) > 0 and 'value_pct_change' in follower_features.columns else 0
            except (KeyError, IndexError) as e:
                # 컬럼이 없거나 데이터가 없는 경우 기본값 사용
                leader_ma6 = 0
                follower_ma6 = 0
                follower_pct_change = 0

            rows.append({
                "b_t": b_t,
                "b_t_1": b_t_1,
                "a_t_lag": a_t_lag,
                "max_corr": corr,
                "best_lag": float(lag),
                "supply_chain_score": supply_chain_score,
                "same_hs4": float(same_hs4),
                "follower_ma6": follower_ma6,
                "follower_pct_change": follower_pct_change,
                "target": b_t_plus_1,
            })

    df_train = pd.DataFrame(rows)
    return df_train


df_train_model = build_training_data(pivot, pairs, df_panel)
print(f'생성된 학습 데이터의 shape: {df_train_model.shape}')
df_train_model.head()



Building training data:   0%|          | 0/1425 [00:00<?, ?it/s]


Building training data:   0%|          | 7/1425 [00:00<00:22, 63.32it/s]


Building training data:   1%|          | 14/1425 [00:00<00:22, 63.62it/s]


Building training data:   1%|▏         | 21/1425 [00:00<00:22, 63.50it/s]


Building training data:   2%|▏         | 28/1425 [00:00<00:21, 63.54it/s]

...


Building training data:  99%|█████████▉| 1414/1425 [00:22<00:00, 61.89it/s]


Building training data: 100%|█████████▉| 1421/1425 [00:22<00:00, 61.90it/s]


Building training data: 100%|██████████| 1425/1425 [00:22<00:00, 62.52it/s]

생성된 학습 데이터의 shape: (54743, 10)


,b_t,b_t_1,a_t_lag,max_corr,best_lag,supply_chain_score,same_hs4,follower_ma6,follower_pct_change,target
0,582317.0,539873.0,14276.0,-0.443984,5.0,2.0,0.0,451077.333333,0.078618,759980.0
1,759980.0,582317.0,52347.0,-0.443984,5.0,2.0,0.0,513740.833333,0.305097,216019.0
2,216019.0,759980.0,53549.0,-0.443984,5.0,2.0,0.0,464275.166667,-0.715757,537693.0
3,537693.0,216019.0,0.0,-0.443984,5.0,2.0,0.0,517713.333333,1.489100,205326.0
4,205326.0,537693.0,26997.0,-0.443984,5.0,2.0,0.0,473534.666667,-0.618135,169440.0


## 5. 회귀 모델 학습


In [6]:
feature_cols = ['b_t', 'b_t_1', 'a_t_lag', 'max_corr', 'best_lag', 
                'supply_chain_score', 'same_hs4', 'follower_ma6', 'follower_pct_change']

train_X = df_train_model[feature_cols].values
train_y = df_train_model["target"].values

# inf, -inf, NaN 값 처리
train_X = np.nan_to_num(train_X, nan=0.0, posinf=0.0, neginf=0.0)
train_y = np.nan_to_num(train_y, nan=0.0, posinf=0.0, neginf=0.0)

# Feature scaling (NMAE 개선을 위해)
scaler = StandardScaler()
train_X_scaled = scaler.fit_transform(train_X)

# 개선 2: NMAE 손실 함수 고려
# NMAE는 상대 오차를 사용하므로, Quantile Regression 고려
USE_QUANTILE = False  # True로 설정하면 Quantile Regression 사용 (중앙값 예측)
RIDGE_ALPHA = 1.0  # 튜닝 가능: 0.1, 1.0, 10.0 등

if USE_QUANTILE:
    # Quantile Regression (중앙값 = 0.5 quantile)
    # NMAE와 유사한 특성: 상대 오차에 덜 민감
    reg = QuantileRegressor(quantile=0.5, alpha=RIDGE_ALPHA, solver='highs')
    print("  - Quantile Regression 사용 (중앙값 예측)")
else:
    # Ridge 회귀 (정규화로 과적합 방지)
    reg = Ridge(alpha=RIDGE_ALPHA)
    print(f"  - Ridge Regression 사용 (alpha={RIDGE_ALPHA})")

reg.fit(train_X_scaled, train_y)

print("Model training completed!")
print(f"Feature importance (coefficients):")
for i, col in enumerate(feature_cols):
    print(f"  {col}: {reg.coef_[i]:.6f}")


  - Ridge Regression 사용 (alpha=1.0)
Model training completed!
Feature importance (coefficients):
  b_t: 3708004.958167
  b_t_1: 1758282.397777
  a_t_lag: -58821.434739
  max_corr: -90863.001956
  best_lag: 31954.255483
  supply_chain_score: 51436.235892
  same_hs4: -6696.114834
  follower_ma6: 7606520.261400
  follower_pct_change: -9527.479585


## 6. 예측 및 제출 파일 생성


In [7]:
def predict(pivot, pairs, reg, df_panel):
    """예측 수행"""
    months = pivot.columns.to_list()
    n_months = len(months)

    t_last = n_months - 1
    t_prev = n_months - 2

    preds = []

    for row in tqdm(pairs.itertuples(index=False), desc="Making predictions", total=len(pairs)):
        leader = row.leading_item_id
        follower = row.following_item_id
        lag = int(row.best_lag)
        corr = float(row.max_corr)
        supply_chain_score = float(row.supply_chain_score)
        same_hs4 = int(row.same_hs4)

        if leader not in pivot.index or follower not in pivot.index:
            continue

        a_series = pivot.loc[leader].values.astype(float)
        b_series = pivot.loc[follower].values.astype(float)

        if t_last - lag < 0:
            continue

        b_t = b_series[t_last]
        b_t_1 = b_series[t_prev]
        a_t_lag = a_series[t_last - lag]

        # 시계열 특성
        current_date = months[t_last]
        try:
            follower_features = df_panel[
                (df_panel['item_id'] == follower) & (df_panel['date'] == current_date)
            ]
            follower_ma6 = follower_features['value_ma6'].values[0] if len(follower_features) > 0 and 'value_ma6' in follower_features.columns else 0
            follower_pct_change = follower_features['value_pct_change'].values[0] if len(follower_features) > 0 and 'value_pct_change' in follower_features.columns else 0
        except (KeyError, IndexError):
            # 컬럼이 없거나 데이터가 없는 경우 기본값 사용
            follower_ma6 = 0
            follower_pct_change = 0

        X_test = np.array([[b_t, b_t_1, a_t_lag, corr, float(lag),
                           supply_chain_score, float(same_hs4), follower_ma6, follower_pct_change]])
        # Feature scaling 적용
        X_test_scaled = scaler.transform(X_test)
        y_pred = reg.predict(X_test_scaled)[0]

        y_pred = max(0.0, float(y_pred))
        y_pred = int(round(y_pred))

        preds.append({
            "leading_item_id": leader,
            "following_item_id": follower,
            "value": y_pred,
        })

    df_pred = pd.DataFrame(preds)
    return df_pred


submission = predict(pivot, pairs, reg, df_panel)
submission.to_csv('../results/submissions/preprocessing_submit.csv', index=False)
print(f"제출 파일 생성 완료: ../results/submissions/preprocessing_submit.csv")
print(f"예측된 공행성쌍 수: {len(submission)}")
submission.head()



Making predictions:   0%|          | 0/1425 [00:00<?, ?it/s]


Making predictions:  24%|██▎       | 338/1425 [00:00<00:00, 3374.12it/s]


Making predictions:  47%|████▋     | 676/1425 [00:00<00:00, 3361.37it/s]


Making predictions:  71%|███████   | 1013/1425 [00:00<00:00, 3279.67it/s]


Making predictions:  94%|█████████▍| 1343/1425 [00:00<00:00, 3286.91it/s]


Making predictions: 100%|██████████| 1425/1425 [00:00<00:00, 3297.35it/s]

제출 파일 생성 완료: ../results/submissions/preprocessing_submit.csv
예측된 공행성쌍 수: 1425


,leading_item_id,following_item_id,value
0,AANGBULD,APQGTRMF,432973
1,AANGBULD,DEWLVASR,609232
2,AANGBULD,DNMPSKTB,5134500
3,AANGBULD,EVBVXETX,4882232
4,AANGBULD,FTSVTTSR,257522
